In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from uproot3_methods.classes.TLorentzVector import TLorentzVector

In [ ]:
file = up.open("../build/evtOutput.root")

In [ ]:
class rootDF:
    def __init__(self, file_name):
        file = up.open(file_name)
        ntp = file["ntp"]
        self.Px = ntp["px"].array()
        self.Py = ntp["py"].array()
        self.Pz = ntp["pz"].array()
        self.E = ntp["E"].array()
        self.nTrk = ntp["nTrk"].array()
        file.close()
    def sign(self, x):
        return -1 if x<0 else 1
    def get_p_from_list(self, iEv, part_list):
        p = TLorentzVector(0, 0, 0, 0)
        for i in part_list:
            if( abs(max(part_list)) >= self.nTrk[iEv]):
                continue
            abs_i = abs(i)
            p += self.sign(i)*TLorentzVector(
                self.Px[iEv][abs_i], self.Py[iEv][abs_i], self.Pz[iEv][abs_i], self.E[iEv][abs_i]
            )
        return p
    def size(self):
        return len(self.Px)
    def apply(self, func, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        return p.func        
    def m2(self, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        return p.mag2
    def m(self, iEv, part_list):
        try:
            p = self.get_p_from_list(iEv, part_list)
            return p.mag
        except ValueError:
            return 0

In [ ]:
df = rootDF('../build/evtOutput.root')

In [ ]:
df.m(9,[0,-1])

In [ ]:
part_list = [2, 3]
m2_list = [df.m2(iEv, part_list) for iEv in range( df.size() )]
m2_list = np.array([m2 for m2 in m2_list if m2])

In [ ]:
part_list = [0, -1]
m_list = [df.m(iEv, part_list) for iEv in range(df.size() )]
m_list = np.array([m for m in m_list if m])

In [ ]:
[m2 for m2 in m2_list if m2<0]

In [ ]:
df.get_p_from_list(0, [2,3])

In [ ]:
df.get_p_from_list(0, [0,-1])

In [ ]:
plt.hist(m2_list, bins=30);

In [ ]:
plt.hist(m_list, bins=30);

In [ ]:
plt.hist(np.sqrt(m2_list), bins = 30);